In [20]:
from thermo import *
from thermo.identifiers import ChemicalMetadataDB
from numpy.testing import assert_allclose
from rdkit import Chem
from rdkit.Chem import Descriptors
from rdkit.Chem.rdMolDescriptors import CalcMolFormula

a = ChemicalMetadataDB(elements=False, main_db=('anion_db_20171009-4.tsv'), user_dbs=[])
a

In [21]:
data = {}
with open('Original metadata.csv') as f:
    f.readline()
    for line in f.readlines():
        if len(line.split('\t')) == 6:
            name, name2, CAS, formula, charge, MW = line.split('\t')
        else:
            name, name2, CAS, formula, charge = line.split('\t')
            MW = 0
        data[CAS] = {'Name': name, 'Name2': name2, 'formula': formula, 'charge':int(charge), 'MW': float(MW)}
# data

In [23]:
good_syns = {CAS:{'synonyms': []} for CAS, d in data.items()}
                  
for CAS, d in data.items():
    # Only take it if original data
    if d['MW']:
        good_syns[CAS]['synonyms'].append(d['Name2'])

for CAS, d in data.items():
    formula = d['formula'].split('-')[0]
    charge = d['charge']
    formulas = [formula+'-'*abs(charge), formula+str(charge),
               formula+'('+'-'+ str(abs(charge)) + ')', formula+'('+ str(abs(charge))+'-' + ')']
#     print(formula, formulas)
    good_syns[CAS]['synonyms'].extend(formulas)

good_syns['338-70-5']['synonyms'].append('oxalate')

import json
f = open('Good synoynms by CAS2.json', 'w')
json.dump(good_syns, f, indent=2, separators=(',', ': '), sort_keys=True)
print('hi')

hi


In [24]:
from collections import Counter
ns = []
for i in a.CAS_index.values():
    ns.extend(list(set(i.all_names)))
Counter(ns).most_common(20)
# No dup names :)
# len(ns)
# len(a.CAS_index), len(a.pubchem_index)
# TODO oxalate goes to the one without Hs, 71081	338-70-5	C2O4-2

[('', 3),
 ('oxalate', 2),
 ('STL483862', 1),
 ('23746-18-1', 1),
 ('Cobaltate(3-),\xc2\xa0hexakis(cyano-\xce\xbaC)-,\xc2\xa0(OC-6-11)-', 1),
 ('1395-EP2284155A2', 1),
 ('1395-EP2316833A1', 1),
 ('3002-EP2314295A1', 1),
 ('Argentate(1-), bis(cyano-C)-', 1),
 ('AC1L18S1', 1),
 ('4607-EP2305673A1', 1),
 ('PdCl6(-2)', 1),
 ('BF4(1-)', 1),
 ('Hydrazoic acid, ion (1-)', 1),
 ('1385048-46-3', 1),
 ('Prussiate', 1),
 ('Thiosulfate (SO3S2-)', 1),
 ('Karbonat', 1),
 ('1395-EP2298776A1', 1),
 ('1395-EP2295434A2', 1)]

In [5]:
# None of the charges are wrong?
for CAS, d in data.items():
    chem = a.search_CAS(CAS)
    if not chem:
        continue
        print('NOTINDB', CAS)
        continue
    mol = Chem.MolFromSmiles(chem.smiles)
    if mol is None:
        continue
        print('CANTREADMOL', CAS)
        continue
#     print(Chem.MolToSmiles(mol))
    charge = Chem.GetFormalCharge(mol)
    try:
        assert charge == d['charge']
#         print('PASS', charge, d['charge'])
    except:
        print('F:', charge, d['charge'], CAS)

In [33]:
# Chem.GetFormalCharge(Chem.MolFromSmiles('[SbH6+3]'))
# a.search_CAS('16971-29-2').InChI, a.search_CAS('16971-29-2').formula, a.search_CAS('16971-29-2').smiles

In [6]:
# mol = Chem.MolFromMolFile('mol/14695-95-5.mol')
# mol = Chem.MolFromMolFile('/tmp/399316.mol')

# # # mol = Chem.MolFromSmiles('[Sb+3]')
# # # When read, 1 atom
# # Chem.MolToSmiles(mol, allHsExplicit=True)
# # mol.GetNumAtoms()
# mw = Descriptors.MolWt(mol)
# formula = CalcMolFormula(mol)
# mw, formula

In [6]:
# Most of the MW ones fail due to having added extra hydrogens???? OR MW?
for CAS, d in data.items():
    chem = a.search_CAS(CAS)
    if not chem:
        continue
    try:
        assert_allclose(chem.MW, d['MW'], atol=0.3)
    except:
        print('F:', CAS, chem.MW, d['MW'],  chem)

# Plenty of outstanding work here and with the charges.
# 20561-39-1 astatine mos stable.
# 16518-47-1 Arsenate (AsO43-), dihydrogen (8CI,9CI), must be a typo - clearly weight is higher.
# 14493-01-7 is fine, confirmed.
# 19469-81-9 H4Al is simply wrong.
# 14897-04-2 is fine
# 34786-97-5 is fine, Vanadate (V(OH)2O21-), their error
# 14333-20-1 Pertechnetate is fine
# 16844-87-4 Arsenate (AsO43-), monohydrogen is fine
# 26450-38-4 their typo for sure
# 15390-83-7 is fine for sure, their bad

('F:', '20561-39-1', 210.0, 211.0, <ChemicalMetadata, name=Astatine ion (At1-), formula=At-, smiles=[At-], MW=210>)
('F:', '16518-47-1', 140.934, 116.95, <ChemicalMetadata, name=Arsenate (AsO43-), dihydrogen (8CI,9CI), formula=H2AsO4-, smiles=O=[As]([O-])(O)O, MW=140.934>)
('F:', '14493-01-7', 514.694, 515.71, <ChemicalMetadata, name=Platinate(2-), tetrabromo-, (SP-4-1)-, formula=Br4Pt-2, smiles=[Br-].[Br-].[Br-].[Br-].[Pt+2], MW=514.694>)
('F:', '19469-81-9', 31.014, 78.84, <ChemicalMetadata, name=Aluminate(1-), tetrahydro- (8CI), formula=H4Al-, smiles=[Al-].[H].[H].[H].[H], MW=31.014>)
('F:', '14897-04-2', 215.041, 216.11, <ChemicalMetadata, name=Cobaltate(3-), hexakis(cyano-κC)-, (OC-6-11)-, formula=C6CoN6-3, smiles=[C-]#N.[C-]#N.[C-]#N.[C-]#N.[C-]#N.[C-]#N.[Co+3], MW=215.041>)
('F:', '34786-97-5', 116.954, 139.94, <ChemicalMetadata, name=Vanadate (V(OH)2O21-), (T-4)- (9CI), formula=H2O4V-, smiles=[O-].[OH-].[OH-].[O].[V+2], MW=116.954>)
('F:', '14333-20-1', 161.996, 163.01, <Chemic